<a href="https://colab.research.google.com/github/allanbrcordeiro/PROJETO-ALURA---IM.IA-2-ed.AL5/blob/main/DOCUMENT_Animais_Pe%C3%A7onhentos_Preven%C3%A7%C3%A3o_e_Primeiros_Socorros_%5BImers%C3%A3o_IA_2%C2%AA_edi%C3%A7%C3%A3o%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-generativeai

In [2]:
#Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

GOOGLE_API_KEY="COLE AQUI A SUA API KEY"
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [3]:
tabela = pd.read_csv("tabela.csv")

In [5]:
df = pd.DataFrame(tabela)
df.columns = ["Animal", "Como Prevenir", "Primeiros Socorros", "Não Fazer"]
df

,Animal,Como Prevenir,Primeiros Socorros,Não Fazer
0,COBRAS/serpentes(Ofidismo),o uso de botas de cano alto ou perneira de cou...,lavar o local da picada apenas com água ou com...,"não fazer torniquete ou garrote, não cortar o ..."
1,Escorpiões/Aranhas (Aracnídeos),manter jardins e quintais limpos. Evitar o acú...,"lavar o local da picada,nusar compressas morna...","não fazer torniquete ou garrote, não cortar o ..."
2,Abelhas e vespas,a remoção das colônias de abelhas e vespas sit...,"em caso de acidente, provocado por múltiplas p...","Não retirar o ferrão com pinças, pois provocam..."


In [6]:
model = "models/embedding-001"

In [7]:
def embed_fn(title, text1, text2, text3):
  content = text1 + " " + text2 + " " + text3  # Concatene os textos
  return genai.embed_content(model=model,
                             content=content,  # Use a nova variável content
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]


In [8]:
df["Embeddings"] = df.apply(lambda row: embed_fn(title=row["Animal"], text1=row["Como Prevenir"], text2=row["Primeiros Socorros"], text3=row["Não Fazer"]), axis=1)
df

,Animal,Como Prevenir,Primeiros Socorros,Não Fazer,Embeddings
0,COBRAS/serpentes(Ofidismo),o uso de botas de cano alto ou perneira de cou...,lavar o local da picada apenas com água ou com...,"não fazer torniquete ou garrote, não cortar o ...","[-0.0073799933, -0.04713731, 0.02066483, 0.022..."
1,Escorpiões/Aranhas (Aracnídeos),manter jardins e quintais limpos. Evitar o acú...,"lavar o local da picada,nusar compressas morna...","não fazer torniquete ou garrote, não cortar o ...","[-0.012924802, -0.042775515, -0.022051811, 0.0..."
2,Abelhas e vespas,a remoção das colônias de abelhas e vespas sit...,"em caso de acidente, provocado por múltiplas p...","Não retirar o ferrão com pinças, pois provocam...","[0.03593755, -0.030663436, 0.0007407885, 0.028..."


In [9]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  colunas_necessarias = ["Animal","Como Prevenir", "Primeiros Socorros", "Não Fazer"]
  return df.iloc[indice][colunas_necessarias]

In [53]:
opcoes_validas = ["cobra","escorpião","Aranha","abelha"]

while True:
  print('Digite uma das opções:', ' | '.join(opcoes_validas))
  consulta = input().lower().strip()

  if consulta in opcoes_validas:
    break
  else:
    print("Opção inválida. Tente novamente.")

trecho = gerar_e_buscar_consulta(consulta, df, model)


Digite uma das opções: cobra | escorpião | Aranha | abelha


KeyboardInterrupt: Interrupted by user

In [27]:
generation_config = {
  "temperature": 0.5,
  "candidate_count": 1
}

safety_settings = [
  {
   "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

In [51]:
prompt = f"Organize esse texto de forma levemente descontraída a fim de acalmar a quem está buscando ajuda, NÃO TIRE e nem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config, safety_settings=safety_settings)
response = model_2.generate_content(prompt)
print(response.text)

**Calma, vamos dar um jeito nas cobras!**

**Como evitar essas bichinhas:**

* Use botas altas ou perneiras de couro, elas são ótimas barreiras.

**Se você for picado:**

* Lave o local com água ou soro fisiológico.
* Não faça torniquete ou corte o local, isso pode piorar as coisas.

**O que NÃO fazer:**

* Não entre em pânico, isso só vai piorar a situação.
* Não tente sugar o veneno, isso não funciona e pode espalhá-lo.
* Não aplique gelo ou calor no local.
